### Load the dependencies

In [2]:
import os
os.getcwd()

'/Users/Apple/Desktop/PhD'

In [76]:
# jayanth = pd.DataFrame({'since_start_hour': [8.0, 8.0, 20.0, 28.0, 37.0, 45.0, 85.0, 85.0],
#                        'Score': [15.20733, 14.02726, 10.86388,11.02701,10.11304,10.01087,9.647795,9.635102]})
# others = pd.DataFrame({'since_start_hour': [0,1,2,3,4,5,6,7],
#                         'BestScore': [9.5,9.4,9.3,9.2,9.1,9.0,8.9,6.5]}) 
# with plt.xkcd():
#     # This figure will be in XKCD-style
#     fig, ax = plt.subplots()
#     ax.plot(others.since_start_hour.values, others.BestScore.values, 'b', alpha=0.8, lw=5, label='others')
#     ax.plot(jayanth.since_start_hour.values, jayanth.Score.values, 'k', alpha=0.8, lw=5, label='jayanth')
#     ax.set_ylim(0.36, 0.4)
#     ax.legend(loc=0)
#     ax.set_xlabel('hours since start')
#     ax.set_ylabel('RMSLE')
#     ax.set_title('Race for the top')
#     plt.show()

In [451]:
# Load the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [452]:
# Read the datasets
train = pd.read_csv('Train.csv').query('ProductCategory == "WomenClothing"') # 84 rows and 4 cols
test = pd.read_csv('template.csv').query('ProductCategory == "WomenClothing"')
weather = pd.read_excel("WeatherData.xlsx")
economic = pd.read_excel("MacroEconomicData.xlsx")
holiday = pd.read_excel("Events_HolidaysData.xlsx")

In [401]:
def print_shape(df):
    print('We have {} rows and {} columns in this dataset.'.format(df.shape[0], df.shape[0]))
    return;

In [453]:
# Check missing values in each data set
train.isnull().sum() # There are 4 missing values in women sales
train
print(train.shape, test.shape)

(84, 4) (12, 4)


In [454]:
def impute(df):
    # Impute using mean
#     df = df.fillna(df.mean())
    # Impute using median    
    df = df.fillna(df.median())
    # Impute using mode
#     df = df.fillna(df.mode())
    #Impute the median for that particular month
    return df
train = impute(train)

In [455]:
# Pre process train dataset for ease
def preprocess(dataset):
    dataset.columns = ['year','month','productcategory','target'] #Renamed the columns
    dataset[['year','month']] = dataset[['year','month']].astype(str)
    dataset['year-month'] = dataset.year.str[:4] + "-" + dataset.month.str[:2] # Add a new column to join later
    dataset = dataset[['target','year-month']]
    return dataset
train = preprocess(train)
test = preprocess(test)

In [456]:
# Cleaned the weatherdata and downloaded onto local.
weather_2009 = pd.read_csv("WeatherData2009.csv")
weather_2010 = pd.read_csv("WeatherData2010.csv")
weather_2011 = pd.read_csv("WeatherData2011.csv")
weather_2012 = pd.read_csv("WeatherData2012.csv")
weather_2013 = pd.read_csv("WeatherData2013.csv")
weather_2014 = pd.read_csv("WeatherData2014.csv")
weather_2015 = pd.read_csv("WeatherData2015.csv")
weather_2016 = pd.read_csv("WeatherData2016.csv")

In [457]:
# Process weather
def process_weather(df):
    # Rename columns for ease
    df.columns = ['year', 'month','day','temp_high','temp_avg','temp_low','dew_point_high','dew_point_avg',
                  'dew_point_low','humidity_high','humidity_avg','humidity_low','pressure_high','pressure_avg','pressure_low',
                 'visibility_high','visibility_avg','visibility_low','wind_low','wind_avg','wind_high','precip_sum','weatherevent']
    # Convert year and day into strings
    df[['year', 'day','month','precip_sum','weatherevent']] = df[['year', 'day','month','precip_sum','weatherevent']].astype(str)
    
    # Convert all other columns to numerics
    df[['temp_high','temp_avg','temp_low','dew_point_high','dew_point_avg',
                  'dew_point_low','humidity_high','humidity_avg','humidity_low','pressure_high','pressure_avg','pressure_low',
                 'visibility_high','visibility_avg','visibility_low','wind_low','wind_avg','wind_high']] = df[['temp_high','temp_avg','temp_low','dew_point_high','dew_point_avg',
                  'dew_point_low','humidity_high','humidity_avg','humidity_low','pressure_high','pressure_avg','pressure_low',
                 'visibility_high','visibility_avg','visibility_low','wind_low','wind_avg','wind_high']].apply(pd.to_numeric, errors = "coerce")
    
    # Fill missing values with median of that column # Try mode/mean
    df = df.fillna(df.median())

    # Scaled all the numerical attributes using range method (simple custom function below)
    df_num = df.select_dtypes(include=[np.number]) # Only include numeric columns
    df_norm = (df_num - df_num.mean()) / (df_num.max() - df_num.min()) # Scale the numeric col
    df[df_norm.columns] = df_norm # Add them back to the original dataframe
    return df

In [458]:
def aggregate_weather(df):
    #Convert year to int
    df['year'] = df['year'].astype(int)
    # Group by month all the weather attributes
    df_aggregate_weather = df.groupby('month').mean() 
    
    #Insert the month columns
    month = ["4", "8", "12", "2", "1","7", "6", "3", "5", "11", "10", "9"] 
    df_aggregate_weather.insert(loc=0, column='month', value=month)
    
    # Convert year back for str
    df_aggregate_weather['year'] = df_aggregate_weather['year'].astype(str)
    
    # Insert a new column 'year-month' that would be useful for future join
    df_aggregate_weather['year-month'] = df_aggregate_weather.year.str[:4] + "-" + df_aggregate_weather.month.str[:2]
    
    # Convert month back to int
    df_aggregate_weather['month'] = df_aggregate_weather['month'].astype(int)
    return df_aggregate_weather

In [459]:
df1 = aggregate_weather(process_weather(weather_2009)) # 12 rows 21 columns
df2 = aggregate_weather(process_weather(weather_2010)) # 12 rows 21 columns
df3 = aggregate_weather(process_weather(weather_2011)) # 12 rows 21 columns
df4 = aggregate_weather(process_weather(weather_2012)) # 12 rows 21 columns
df5 = aggregate_weather(process_weather(weather_2013)) # 12 rows 21 columns
df6 = aggregate_weather(process_weather(weather_2014)) # 12 rows 21 columns
df7 = aggregate_weather(process_weather(weather_2015)) # 12 rows 21 columns
df8 = aggregate_weather(process_weather(weather_2016)) # 12 ros 21 columns
# clean_weather(weather_2016)

In [514]:
# Merge with the train set

df_new = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])
train_weather = train.merge(df_new, on='year-month', how = 'left')
test_weather = test.merge(df_new, on='year-month', how = 'left')

In [666]:
train_weather #184 ros 22 cols
test_weather #12 rows 22 cols

,target,year-month,month,year,temp_high,temp_avg,temp_low,dew_point_high,dew_point_avg,dew_point_low,...,humidity_low,pressure_high,pressure_avg,pressure_low,visibility_high,visibility_avg,visibility_low,wind_low,wind_avg,wind_high
0,119,2016-1,1,2016,-0.294063,-0.274287,-0.269134,-0.254580,-0.259487,-0.258387,...,-0.036636,-0.005108,-0.026409,-0.051023,-0.015865,0.044979,0.102013,0.032131,0.113643,0.094656
1,151,2016-2,2,2016,-0.244453,-0.236081,-0.235764,-0.193934,-0.209998,-0.237586,...,-0.033753,0.034770,-0.019280,-0.065458,0.016393,-0.011676,-0.014645,0.037535,0.138308,0.126457
2,72,2016-3,3,2016,-0.087612,-0.100374,-0.116446,-0.111999,-0.127924,-0.145484,...,-0.124921,0.019086,0.001450,-0.020265,0.016393,0.008420,0.015319,0.039839,0.086995,0.115969
3,78,2016-4,4,2016,-0.028974,-0.046401,-0.067820,-0.069934,-0.103214,-0.140667,...,-0.167918,0.014167,-0.000994,-0.013814,0.016393,0.086485,0.149795,0.034595,0.063245,0.024014
4,130,2016-5,5,2016,0.070094,0.067227,0.064199,0.046711,0.053606,0.063548,...,0.036369,-0.056720,-0.023476,0.009742,-0.011832,-0.028139,-0.025003,-0.027532,-0.068685,-0.065482
5,190,2016-6,6,2016,0.199915,0.181136,0.168476,0.139399,0.143191,0.147333,...,-0.013532,-0.104167,-0.073721,-0.053349,-0.012773,-0.033515,-0.056455,-0.019682,-0.072986,-0.059319
6,200,2016-7,7,2016,0.267227,0.260074,0.264199,0.235743,0.251582,0.275806,...,0.066505,-0.095430,-0.055734,-0.028517,-0.040058,-0.099107,-0.176213,-0.029531,-0.102346,-0.075275
7,166,2016-8,8,2016,0.272245,0.266386,0.272801,0.242840,0.255377,0.284194,...,0.074145,-0.017204,0.015380,0.037499,0.008329,0.010571,-0.047181,-0.039522,-0.110761,-0.100045
8,62,2016-9,9,2016,0.179174,0.174614,0.181809,0.171399,0.181099,0.196667,...,0.079451,0.025000,0.054309,0.079210,-0.004440,0.004262,-0.018955,-0.020272,-0.033856,-0.059319
9,132,2016-10,10,2016,0.014897,0.018840,0.024055,0.062195,0.062461,0.068710,...,0.111072,0.067473,0.063034,0.061505,0.008329,-0.010935,-0.018955,-0.018968,-0.051855,-0.043017


In [ ]:
# Xtr, Xv, ytr, yv = train_test_split(train[feature_names].values, y, test_size=0.2, random_state=1987)
# dtrain = xgb.DMatrix(Xtr, label=ytr)
# dvalid = xgb.DMatrix(Xv, label=yv)
# dtest = xgb.DMatrix(test[feature_names].values)
# watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

# # Try different parameters! My favorite is random search :)
# xgb_pars = {'min_child_weight': 50, 'eta': 0.3, 'colsample_bytree': 0.3, 'max_depth': 10,
#             'subsample': 0.8, 'lambda': 1., 'nthread': 4, 'booster' : 'gbtree', 'silent': 1,
#             'eval_metric': 'rmse', 'objective': 'reg:linear'}

In [518]:
X = train_weather.drop(['year-month','year'], axis =1).iloc[:,1:]
Y = train_weather[['target']] #Y = train["target"]

In [519]:
test_weather.drop(['year-month','year','target'], axis =1)
test_weather

,target,year-month,month,year,temp_high,temp_avg,temp_low,dew_point_high,dew_point_avg,dew_point_low,...,humidity_low,pressure_high,pressure_avg,pressure_low,visibility_high,visibility_avg,visibility_low,wind_low,wind_avg,wind_high
0,119,2016-1,1,2016,-0.294063,-0.274287,-0.269134,-0.254580,-0.259487,-0.258387,...,-0.036636,-0.005108,-0.026409,-0.051023,-0.015865,0.044979,0.102013,0.032131,0.113643,0.094656
1,151,2016-2,2,2016,-0.244453,-0.236081,-0.235764,-0.193934,-0.209998,-0.237586,...,-0.033753,0.034770,-0.019280,-0.065458,0.016393,-0.011676,-0.014645,0.037535,0.138308,0.126457
2,72,2016-3,3,2016,-0.087612,-0.100374,-0.116446,-0.111999,-0.127924,-0.145484,...,-0.124921,0.019086,0.001450,-0.020265,0.016393,0.008420,0.015319,0.039839,0.086995,0.115969
3,78,2016-4,4,2016,-0.028974,-0.046401,-0.067820,-0.069934,-0.103214,-0.140667,...,-0.167918,0.014167,-0.000994,-0.013814,0.016393,0.086485,0.149795,0.034595,0.063245,0.024014
4,130,2016-5,5,2016,0.070094,0.067227,0.064199,0.046711,0.053606,0.063548,...,0.036369,-0.056720,-0.023476,0.009742,-0.011832,-0.028139,-0.025003,-0.027532,-0.068685,-0.065482
5,190,2016-6,6,2016,0.199915,0.181136,0.168476,0.139399,0.143191,0.147333,...,-0.013532,-0.104167,-0.073721,-0.053349,-0.012773,-0.033515,-0.056455,-0.019682,-0.072986,-0.059319
6,200,2016-7,7,2016,0.267227,0.260074,0.264199,0.235743,0.251582,0.275806,...,0.066505,-0.095430,-0.055734,-0.028517,-0.040058,-0.099107,-0.176213,-0.029531,-0.102346,-0.075275
7,166,2016-8,8,2016,0.272245,0.266386,0.272801,0.242840,0.255377,0.284194,...,0.074145,-0.017204,0.015380,0.037499,0.008329,0.010571,-0.047181,-0.039522,-0.110761,-0.100045
8,62,2016-9,9,2016,0.179174,0.174614,0.181809,0.171399,0.181099,0.196667,...,0.079451,0.025000,0.054309,0.079210,-0.004440,0.004262,-0.018955,-0.020272,-0.033856,-0.059319
9,132,2016-10,10,2016,0.014897,0.018840,0.024055,0.062195,0.062461,0.068710,...,0.111072,0.067473,0.063034,0.061505,0.008329,-0.010935,-0.018955,-0.018968,-0.051855,-0.043017


In [ ]:
# Ecocnomic Data

In [636]:
def process_economic(df):
    # Rename columns for ease
    df.columns = ['year-month', 'gdp','realgdp','cpi','party',
                  'unemployment','interestrate',
                  'financerate','wagesperhour','adv',
                  'cotton','cottonchange','planted','harvested','yield',
                 'production','usage','exports']
    # Convert year into strings
    df[['year-month']] = df[['year-month']].astype(str)
    
    # Drop Adversitising, Party
    df = df.drop(['adv','party'], axis=1)
    # Convert all other columns to numerics
    df[['gdp','realgdp','cpi','unemployment','interestrate',
       'financerate','wagesperhour','cotton','cottonchange','planted','harvested','yield',
                 'production','usage','exports']] = df[['gdp','realgdp','cpi','unemployment','interestrate',
       'financerate','wagesperhour','cotton','cottonchange','planted','harvested','yield',
                 'production','usage','exports']].apply(pd.to_numeric, errors = "coerce")
    
    # Fill missing values with median of that column # Try mode/mean
    df = df.fillna(df.median())

    # Scaled all the numerical attributes using range method (simple custom function below)
    df_num = df.select_dtypes(include=[np.number]) # Only include numeric columns
    df_norm = (df_num - df_num.mean()) / (df_num.max() - df_num.min()) # Scale the numeric col
    df[df_norm.columns] = df_norm # Add them back to the original dataframe
    return df

In [741]:
# Merging Economic Data with train_weather

train_weather_economic = pd.concat([train_weather, test_weather], axis=0)
train_weath_econ = pd.concat([train_weather, process_economic(economic.iloc[0:84,:])], axis=1)
train_weath_econ = train_weath_econ.drop(['year-month','year'],axis=1) # 84 rows & 36 cols
train_weath_econ
# # Merging Economic Data with test_weather
test_weath_econ = pd.read_csv("test_weather_economic.csv") # 12 rows 36 cols
test_weath_econ = test_weath_econ.drop(['year-month','year'], axis=1) #12 rows & 35 cols
test_weath_econ


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x117b0c588>>
Traceback (most recent call last):
  File "/Users/Apple/anaconda/lib/python3.6/site-packages/xgboost-0.6-py3.6.egg/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


,month,temp_high,temp_avg,temp_low,dew_point_high,dew_point_avg,dew_point_low,humidity_high,humidity_avg,humidity_low,...,financerate,wagesperhour,cotton,cottonchange,planted,harvested,yield,production,usage,exports
0,1,-0.294063,-0.274287,-0.269134,-0.254580,-0.259487,-0.258387,-0.171988,-0.102917,-0.036636,...,-0.086207,-0.239247,-0.350214,-0.272094,-0.536704,-0.501495,-0.375758,-0.386950,0.166667,-0.217742
1,2,-0.244453,-0.236081,-0.235764,-0.193934,-0.209998,-0.237586,-0.072973,-0.053764,-0.033753,...,0.551724,-0.255376,-0.489957,-0.338973,-0.536704,-0.501495,-0.375758,-0.386950,0.166667,-0.370968
2,3,-0.087612,-0.100374,-0.116446,-0.111999,-0.127924,-0.145484,-0.065928,-0.100579,-0.124921,...,0.551724,-0.255376,-0.561111,-0.219546,-0.536704,-0.513666,-0.339394,-0.386950,0.166667,-0.370968
3,4,-0.028974,-0.046401,-0.067820,-0.069934,-0.103214,-0.140667,-0.083474,-0.132541,-0.167918,...,0.551724,-0.077957,-0.316239,0.378384,-0.536704,-0.513666,-0.412121,-0.404859,0.166667,-0.379032
4,5,0.070094,0.067227,0.064199,0.046711,0.053606,0.063548,0.027425,0.032660,0.036369,...,-0.103448,0.051075,-0.252137,0.028065,-0.536704,-0.502776,-0.430303,-0.400909,0.166667,-0.548387
5,6,0.199915,0.181136,0.168476,0.139399,0.143191,0.147333,0.018546,0.000792,-0.013532,...,-0.103448,-0.368280,-0.007265,0.346537,0.068663,-0.074845,0.242424,-0.477285,0.166667,-0.080645
6,7,0.267227,0.260074,0.264199,0.235743,0.251582,0.275806,0.104160,0.085489,0.066505,...,-0.103448,-0.190860,0.438889,0.661027,0.380836,0.255712,0.424242,0.325981,0.166667,0.241935
7,8,0.272245,0.266386,0.272801,0.242840,0.255377,0.284194,0.088031,0.085956,0.074145,...,-0.120690,-0.206989,0.387607,-0.165406,0.380836,0.406897,0.151515,0.352054,0.166667,0.241935
8,9,0.179174,0.174614,0.181809,0.171399,0.181099,0.196667,0.098849,0.090647,0.079451,...,-0.120690,0.147849,0.233761,-0.324642,0.463296,0.486334,0.206061,0.422110,-0.166667,0.241935
9,10,0.014897,0.018840,0.024055,0.062195,0.062461,0.068710,0.127132,0.122890,0.111072,...,-0.120690,0.631720,0.276068,-0.018909,0.463296,0.486334,0.115152,0.393666,-0.166667,0.395161


In [742]:
# Create X and Y for weather_economic merge
X = train_weath_econ.iloc[:,1:] # 84 rows 35 cols
Y = train_weath_econ[['target']] # 84 rows 1 cols

In [524]:
# Modelling
#Split the data

x_train = X.iloc[0:72,:]
x_val = X.iloc[72:,:]
y_train = Y.iloc[0:72,:]
y_val = Y.iloc[72:,:]
y_val

dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_val, label=y_val)
# dtest = xgb.DMatrix(test[feature_names].values)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

In [743]:
# Simple Xgboost
import xgboost as xgb
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X, Y)

# y_test_xgb = model.predict(test_weather.drop(['year-month','year','target'], axis =1))
y_test_xgb = model.predict(test_weath_econ)
results_xgb = pd.DataFrame(data={'y':y_test}) 
results_xgb

,y
0,3350.0
1,2620.0
2,1755.0
3,3350.0
4,2164.0
5,2220.0
6,2164.0
7,2371.0
8,2919.0
9,3800.0


In [754]:
# Xgboost with parameter tuning
from sklearn.model_selection import GridSearchCV

cv_params = {'max_depth': [3,4,5,6,7,8],
#              'subsample': [0.5,0.6,0.7,0.8],
                          'subsample': [0.5,0.6],
#              'learning_rate': [0.001,0.01,0.05,0.3,0.5,1],
            'learning_rate': [0.01,0.3],
#             'gamma':[i/10.0 for i in range(3,6)],
            'colsample_bytree': [0.3], 
             'min_child_weight':[4,5]}
ind_params = {'n_estimators': 50, 'seed':10, 
             'objective': 'reg:linear'}
optimized_GBM = GridSearchCV(xgb.XGBRegressor(**ind_params), 
                            cv_params,
                            cv = 5, n_jobs = 3)
optimized_GBM.fit(X, Y)

y_test_xgb_cv = optimized_GBM.best_estimator_.predict(test_weath_econ)
results_xgb_cv = pd.DataFrame(data={'y':y_test_xgb_cv}) 
results_xgb_cv

,y
0,1878.261230
1,2416.914307
2,3181.469727
3,3467.224121
4,3376.289307
5,3100.838135
6,3378.258789
7,3942.145996
8,4126.750977
9,3953.855469


In [550]:
FOREVER_COMPUTING_FLAG = True
xgb_pars = []
for MCW in [10, 20]:
    for ETA in [0.05, 0.1, 0.15]:
        for CS in [0.3]:
            for MD in [6, 8, 10]:
                for SS in [0.5, 0.6]:
                    for LAMBDA in [0.5]:
                        xgb_pars.append({'min_child_weight': MCW, 'eta': ETA, 
                                         'colsample_bytree': CS, 'max_depth': MD,
                                         'subsample': SS, 'lambda': LAMBDA, 
                                         'nthread': -1, 'booster' : 'gbtree', 'eval_metric': 'rmse',
                                         'silent': 1, 'objective': 'reg:linear'})

while FOREVER_COMPUTING_FLAG:
    xgb_par = np.random.choice(xgb_pars, 1)[0]
    print(xgb_par)
    model = xgb.train(xgb_par, dtrain, 20, watchlist, early_stopping_rounds=50,
                      maximize=False, verbose_eval=100)
    print('Modeling RMSLE %.5f' % model.best_score)

{'min_child_weight': 10, 'eta': 0.05, 'colsample_bytree': 0.3, 'max_depth': 10, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3235.68	valid-rmse:3975.47
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:1389.54	valid-rmse:2100.56
Modeling RMSLE 2100.56201
{'min_child_weight': 20, 'eta': 0.15, 'colsample_bytree': 0.3, 'max_depth': 8, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:2933.48	valid-rmse:3667.94
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:747.172	valid-rmse:1102.47
Modeling RMSLE 1102.47266
{'min_child_weight': 20, 'eta': 0.05, 'colsample_bytree': 0.3, 'm

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:861.801	valid-rmse:1334.27
Modeling RMSLE 1334.27246
{'min_child_weight': 20, 'eta': 0.15, 'colsample_bytree': 0.3, 'max_depth': 8, 'subsample': 0.5, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:2940.57	valid-rmse:3675.18
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:776.095	valid-rmse:1091.45
Modeling RMSLE 1089.90466
{'min_child_weight': 20, 'eta': 0.1, 'colsample_bytree': 0.3, 'max_depth': 8, 'subsample': 0.5, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3089.07	valid-rmse:3826.44
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	trai

Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:1389.54	valid-rmse:2100.56
Modeling RMSLE 2100.56201
{'min_child_weight': 10, 'eta': 0.1, 'colsample_bytree': 0.3, 'max_depth': 8, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3084.32	valid-rmse:3821.61
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:723.225	valid-rmse:1353.29
Modeling RMSLE 1353.29163
{'min_child_weight': 20, 'eta': 0.15, 'colsample_bytree': 0.3, 'max_depth': 10, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:2933.48	valid-rmse:3667.94
Multiple eval metrics have been passed: 'valid-rmse' will be used 

Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:545.452	valid-rmse:1050.68
Modeling RMSLE 1050.68164
{'min_child_weight': 10, 'eta': 0.05, 'colsample_bytree': 0.3, 'max_depth': 8, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3235.68	valid-rmse:3975.47
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:1389.54	valid-rmse:2100.56
Modeling RMSLE 2100.56201
{'min_child_weight': 20, 'eta': 0.15, 'colsample_bytree': 0.3, 'max_depth': 10, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:2933.48	valid-rmse:3667.94
Multiple eval metrics have been passed: 'valid-rmse' will be used

Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:847.916	valid-rmse:1345.13
Modeling RMSLE 1345.13342
{'min_child_weight': 10, 'eta': 0.15, 'colsample_bytree': 0.3, 'max_depth': 8, 'subsample': 0.5, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:2940.57	valid-rmse:3675.18
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:592.235	valid-rmse:1051.79
Modeling RMSLE 1047.44092
{'min_child_weight': 10, 'eta': 0.05, 'colsample_bytree': 0.3, 'max_depth': 8, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3235.68	valid-rmse:3975.47
Multiple eval metrics have been passed: 'valid-rmse' will be used 

...

Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:847.916	valid-rmse:1345.13
Modeling RMSLE 1345.13342
{'min_child_weight': 20, 'eta': 0.1, 'colsample_bytree': 0.3, 'max_depth': 8, 'subsample': 0.5, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3089.07	valid-rmse:3826.44
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:861.801	valid-rmse:1334.27
Modeling RMSLE 1334.27246
{'min_child_weight': 20, 'eta': 0.15, 'colsample_bytree': 0.3, 'max_depth': 6, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:2933.48	valid-rmse:3667.94
Multiple eval metrics have been passed: 'valid-rmse' will be used f

Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:861.801	valid-rmse:1334.27
Modeling RMSLE 1334.27246
{'min_child_weight': 10, 'eta': 0.05, 'colsample_bytree': 0.3, 'max_depth': 10, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3235.68	valid-rmse:3975.47
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:1389.54	valid-rmse:2100.56
Modeling RMSLE 2100.56201
{'min_child_weight': 10, 'eta': 0.1, 'colsample_bytree': 0.3, 'max_depth': 8, 'subsample': 0.6, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:3084.32	valid-rmse:3821.61
Multiple eval metrics have been passed: 'valid-rmse' will be used 

Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:776.095	valid-rmse:1091.45
Modeling RMSLE 1089.90466
{'min_child_weight': 20, 'eta': 0.15, 'colsample_bytree': 0.3, 'max_depth': 6, 'subsample': 0.5, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:2940.57	valid-rmse:3675.18
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:776.095	valid-rmse:1091.45
Modeling RMSLE 1089.90466
{'min_child_weight': 20, 'eta': 0.15, 'colsample_bytree': 0.3, 'max_depth': 10, 'subsample': 0.5, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:2940.57	valid-rmse:3675.18
Multiple eval metrics have been passed: 'valid-rmse' will be used

Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[19]	train-rmse:747.172	valid-rmse:1102.47


KeyboardInterrupt: 